# Recommenders 3 -- Sequence Recommenders (45m) 

## Goals of this practical:

- Understand the sequence recommendation framework (~5min)
- Load/Format dataset (~5min)
- Understand/train the prod2vec model (~10min)
- Evaluate (~10min)
- Visualize (~10min)
- Fiddle (~5min)



In [ ]:
#! pip install gensim --upgrade

In [2]:
import pandas as pd
import numpy as np

# Sequence Recommenders:

> What will you click next ?

The sequence recommendation setting is a particular case of the implicit collaborative filtering setting. Given a sequence of items $i_0,i_1,...,i_n$ the goal is to predict the $i_{(n+1)},...$ items the user will consume. Playlist continuation is a neat use case of sequence recommenders. You've been listening to those songs, what can you listen to now ?


This setting differs from the classical collaborative filtering because the history is the recent trace and not the full saved interactions. Also, it's possible to do sequence recommendation without any specific latent user profile. 

#### Here we propose to explore this unpersonalized sequence recommandation

## Data used : [smallest movie-lens dataset](https://grouplens.org/datasets/movielens/)

Here we'll use the same data as before but instead of seeing $(user,item,rating)$ triplets or a $(user,item)$ interaction , we'll see item sequences: $user: [item, item,...]$

## Loading Data (same as before but in chronological order):

In [3]:
ratings = pd.read_csv("dataset/ratings.csv")
ratings = ratings.sort_values("timestamp",ascending=True)
print(ratings.iloc[0]["timestamp"] < ratings.iloc[-1]["timestamp"] ) # just checking 

ratings.head(5)

# we also load titles and create an id2title dictionnary
titleCSV = pd.read_csv("dataset/movies.csv")
titleCSV.head(5)
id2title = titleCSV[["movieId","title"]].set_index("movieId").to_dict()["title"]

True


## (a) Create sequence datasets:
For this task, we need sequences of items as data:

## (Todo): extract all movie sequences (in chronological order) from the dataset:


In this dataset, each user has seen at least 20 movies.


- We need to extract all movie rating sequences (there is one per user) from the dataset:

`sequence_of_movies = [[movieid,...],[movieid,...],...]`

In [5]:
sequences_of_movies = [list(ratings[ratings['userId'] == id]['movieId'].values) for id in ratings['userId'].unique()]

In [7]:
#sequences_of_movies

## (Todo): Create a train/test dataset

Here, we propose as task to predict the last 5 items of each sequence.

In [8]:
train_seq,test_seq = [],[]

for seq in sequences_of_movies:
    train_seq.append(seq[:-5])
    test_seq.append(seq[-5:])
    
last_consumed_item = [seq[-1] for seq in train_seq] # We save the last consumed item for each list
                                                    # We'll use it as a starting point

## (Todo): Create the list of the most popular movies

- Here, popular is the number of times the movie appears in a list

In [9]:
from collections import Counter

counts = np.asarray([Counter(seq) for seq in sequences_of_movies]).sum()
most_popular = list(dict(counts.most_common()).keys())
num_items = len(most_popular)

In [11]:
#most_popular

``` python
#Most popular looks like this:
[356,318,296,2571,593,260,480,110,589,...]
 ```

## Word2Vec skip-gram <=> Prod2Vec


### Word2Vec

The MAIN idea of word2vec is to maximise the similarity (dot product) between the vectors for words which appear close together (in the context of each other) in text, and minimise the similarity of words that do not. 

This can be applied to products instead of words: it clusters similar products together.


#### Paper Abstract:
> In recent years online advertising has become increasingly ubiquitous and effective. Advertisements shown to visitors fund sites and apps that publish digital content, manage social networks, and operate e-mail services. Given such large variety of internet resources, determining an appropriate type of advertising for a given platform has become critical to financial success. Native advertisements, namely ads that are similar in look and feel to content, have had great success in news and social feeds. However, to date there has not been a winning formula for ads in e-mail clients. In this paper we describe a system that leverages user purchase history determined from e-mail receipts to deliver highly personalized product ads to Yahoo Mail users. We propose to use a novel neural language-based algorithm specifically tailored for delivering effective product recommendations, which was evaluated against baselines that included showing popular products and products predicted based on co-occurrence. We conducted rigorous offline testing using a large-scale product purchase data set, covering purchases of more than 29 million users from 172 e-commerce websites. Ads in the form of product recommendations were successfully tested on online traffic, where we observed a steady 9% lift in click-through rates over other ad formats in mail, as well as comparable lift in conversion rates. Following successful tests, the system was launched into production during the holiday season of 2014

[Prod2Vec Model](https://arxiv.org/abs/1606.07154)



## Gensim has the best python implementation of word2vec's algorithms:

We can just use these raw implementations. The only thing to do is to consider items as words:

In [13]:
pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\hider\anaconda3\python.exe -m pip install --upgrade pip' command.


In [19]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



train_seq_str = [list(map(str,seq)) for seq in train_seq] # we just say that our items id's are strings..
    

# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=train_seq_str,
                                 window=10,               ### here we train a cbow model 
                                min_count=0,                      
                                sample=0.001, ns_exponent=0.75, workers=10,
                                sg=1, hs=0, negative=15,          ### set sg to 1 to train a sg model => Prod2Vec
                                cbow_mean=0)

2022-04-08 04:09:20,581 : INFO : collecting all words and their counts
2022-04-08 04:09:20,582 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-04-08 04:09:20,612 : INFO : collected 9616 word types from a corpus of 97786 raw words and 610 sentences
2022-04-08 04:09:20,614 : INFO : Creating a fresh vocabulary
2022-04-08 04:09:20,680 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=0 retains 9616 unique words (100.0%% of original 9616, drops 0)', 'datetime': '2022-04-08T04:09:20.680216', 'gensim': '4.1.2', 'python': '3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
2022-04-08 04:09:20,681 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=0 leaves 97786 word corpus (100.0%% of original 97786, drops 0)', 'datetime': '2022-04-08T04:09:20.681216', 'gensim': '4.1.2', 'python': '3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]', 'pla

2022-04-08 04:09:29,685 : INFO : Word2Vec lifecycle event {'msg': 'training on 488930 raw words (488340 effective words) took 8.8s, 55792 effective words/s', 'datetime': '2022-04-08T04:09:29.685896', 'gensim': '4.1.2', 'python': '3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
2022-04-08 04:09:29,687 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=9616, vector_size=100, alpha=0.025)', 'datetime': '2022-04-08T04:09:29.687895', 'gensim': '4.1.2', 'python': '3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


### A few things:

In [23]:
print(w2v.wv.vectors[0])              # The vector of index 0
print(w2v.wv.index_to_key[0])           # codes for the movieId 356
print(w2v.wv.vocab["356"].index)      # Inverse mapping


[ 0.02438856  0.30555177  0.06609589  0.26134652  0.15534046 -0.15790056
  0.04711282  0.19295576 -0.05538234 -0.07068532  0.41293344 -0.3753956
  0.26052552  0.6414371   0.1288608  -0.36397532  0.39084393 -0.46268812
  0.2717409   0.09429979  0.02121332  0.16194539  0.14519459 -0.12208986
  0.11794051  0.00280935 -0.23812935  0.2395958  -0.5385449  -0.02966912
  0.7163183  -0.5792978   0.13287121 -0.21727818 -0.17784558  0.09210565
  0.4342135   0.744411   -0.16502066 -0.12780556 -0.88974434 -0.2509973
 -0.88379526 -0.13419028 -0.30140287 -0.15829225  0.3491251  -0.0225413
 -0.03926184 -0.25274512 -0.1106815   0.264428    0.1311739  -0.14145315
 -0.09879348 -0.3598533   0.5893788  -0.18960936 -0.34112883  0.55977416
 -0.3113862   0.22315863  0.18040575 -0.00585445 -0.36261898  0.50615543
  0.14591226 -0.02343903 -0.00109     0.3896734   0.5369442   0.37874386
  0.832242    0.33986717  0.04857073  0.12060176  0.29804763  0.074908
  0.2757374  -0.5777223  -0.53545463  0.19060588  0.3371

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

## Getting similar items:

The heart of the algorithm is in the similar item search. As in word2vec, we simply use cosine distance between items to find "similar items"

### We can search by id's

In [24]:
def get_similar_ids(w2vmodel,iid,num=5):
    
    if str(iid) in w2vmodel.wv.vocab:
        return [int(iid) for iid,_ in w2vmodel.wv.most_similar(str(iid),topn=num)] 
    else:
        return []

get_similar_ids(w2v,last_consumed_item[0],num=5)

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

### Or by vector

In [ ]:
w2v.wv[str(last_consumed_item[0])]

In [ ]:
def get_similar_vectors(w2vmodel,vec,num=5):
        return [int(iid) for iid,_ in w2vmodel.wv.most_similar(positive=[vec],topn=num)] 

get_similar_vectors(w2v,w2v.wv[str(last_consumed_item[0])],num=5) # items are strings

### Let's see if this works

We can query by id

In [ ]:
ID = 1
NUM_SIM = 3

print("Movies similar to: ", id2title[ID])
print("")
for x in get_similar_ids(w2v,ID,NUM_SIM):
    print("--> ",id2title[x])

We can also query by vector

**NOTE:** the 1st results can be the item(s) you've used to query

In [ ]:
ID = 1
NUM_SIM = 4
print("Movies similar to: ", id2title[ID])
print("")
for x in get_similar_vectors(w2v,w2v.wv[str(ID)],NUM_SIM):
    print("--> ",id2title[x])

Our result was the following (for ID = 1 & NUM_SIM = 3)

>Movies similar to:  Toy Story (1995)
>-  Beauty and the Beast (1991)
-   Toy Story 2 (1999)
-   Lion King, The (1994)


Using vectors enables operations like additions to be made

In [ ]:
ID1 = 2571
ID2 = 589
NUM_SIM = 10

vec = np.max([w2v.wv[str(ID1)],w2v.wv[str(ID2)]],axis=0)# + w2v.wv[str(318)]

print("Movies similar to: ", id2title[ID1] , "+",  id2title[ID2] )
print("")
for x in get_similar_vectors(w2v,vec ,NUM_SIM):
    print("--> ",id2title[x])

Ok, we now have a good base for our sequence recommendation algorithm, let's write something to evaluate our predictions

## (Todo) write a `get_relevance_list(proposed_ids,real_ids)` function:

This function will be used to compare proposed items w/ real items:


- A relevant item is an item which is in the ground truth
- It returns a list which length is the number of proposed items filled of 0's and 1's : 0 means the item is not relevant, 1 means it's relevant.

- get_relevance_list([1,2,3,4],[1,4,5,6]) should returns [1,0,0,1]  because items 1 and 4 are relevant.


In [ ]:
def get_relevance_list(proposed_ids,real_ids):
    real_ids = set(real_ids)
    return #To Complete
get_relevance_list([1,2,3,4],[1,4,5,6]) #returns [1,0,0,1]

### Let's test our function on our data

In [ ]:
get_relevance_list(most_popular[:25],test_seq[1])

In [ ]:
get_relevance_list(get_similar_ids(w2v,last_consumed_item[0],25),test_seq[0])

## Ok, now, let's write prediction funtions:

- `predict_pop` will recommend the k's most popular items
- `predict_w2v` will recommend the k's most similar items to the last one consumed

#### (TODO) : complete those functions

In [ ]:
def predict_pop(last_seen,k):
    return #To Complete

def predict_w2v(last_seen,k):
    return #To Complete

#data is list of last_consumed:
def get_predictions(predict_func,data,truth,k=5):
    if k == -1 or k == 0:
        k = num_items
    return [get_relevance_list(predict_func(last_seen,k),will_see) for last_seen,will_see in zip(data,truth)]

**Note**: The `get_predictions(...)` function returns the relevant list associated to predictions

### The following cells should return list of lists

In [ ]:
print(get_predictions(predict_pop,last_consumed_item[:5],test_seq[:5],3))
print(get_predictions(predict_w2v,last_consumed_item[:5],test_seq[:5],3))

expected output: 
```
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]

```

## The return of the MRR and nDCG functions

In [ ]:
test_list = [[0,0,1],[0,1,0],[1,0,0],[0,0,0]]

def rr(list_items):
    relevant_indexes = np.asarray(list_items).nonzero()[0]
    
    if len(relevant_indexes) > 0:
        return 1/(relevant_indexes[0]+1) # arrays are indexed from 0
    else:
        return 0

def mrr(list_list_items):
    return np.mean([rr(list_item) for list_item in list_list_items])

mrr(test_list) #0.4583333333333333

# The dcg@k is the sum of the relevance, penalized gradually
def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        
    """
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        
    return 0.

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# dcg_at_k(r, 1) => 3.0
# dcg_at_k(r, 2) => 4.2618595071429155
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
print(dcg_at_k(r, 1))
print(dcg_at_k(r, 2))

def mean_dcg(rel_lists,k):
    return np.mean([dcg_at_k(rel_list,k) for rel_list in rel_lists])

# And it's normalized version
def ndcg_at_k(r, k):
    """
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    """
    dcg_max =  dcg_at_k(sorted(r)[::-1],k) 
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# ndcg_at_k(r, 1) => 1.0
# ndcg_at_k(r, 4) => 0.794285
    
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]    
ndcg_at_k(r, 4)

def mean_ndcg(rel_lists,k):
    return np.mean([ndcg_at_k(rel_list,k) for rel_list in rel_lists])

## Let's see how this naïve way of predicting items to show works

In [ ]:
pop_preds = get_predictions(predict_pop,last_consumed_item,test_seq,-1)
w2v_preds = get_predictions(predict_w2v,last_consumed_item,test_seq,-1)

print("1/MRR")
print(1/mrr(pop_preds))
print(1/mrr(w2v_preds))
print("")
print("DCG")
print(mean_dcg(pop_preds,5))
print(mean_dcg(w2v_preds,5))
print("")
print("nDCG")
print(mean_ndcg(pop_preds,5))
print(mean_ndcg(w2v_preds,5))

### (TODO) Can we do better ?

Now, try a different strategy: 


- History should be discarded from prediction
- Instead of basing the prediction on the last seen item, we'll take all the `seen[-n:]` ones (horizon) into account
- To aggregate all items, we'll simply take the min rank to take into account the history offset.
- Equal scores can be handled using the history offset.
Example: 

> Let's say you chose to use the two last seen items `[item 44, Item 398]` to predict the following items

Therefore, using `get_similar_ids` method on both items will yield two lists of **similar** ranked item id's:
 - Similar to item 44: `[item 1, item 33, item 5]`
 - Similar to item 398: `[item 25, item 1, item 5]`
 scores (rank,offset):
 ```
 scores := {item 1: (0,0), item 33: (1,0), item 5: (2,0) , item 25: (0,1)}```
 
 
Then, aggregation by best rank should yield: `[1,25,33,5]`


In [ ]:
def predict_max_w2v(seen,k,horizon=2):
    
    # To complete
    
    return # a list


w2v_best_preds = get_predictions(predict_max_w2v,train_seq,test_seq,-1)

print(1/mrr(w2v_max_preds))
print(mean_dcg(w2v_max_preds,5))
print(mean_ndcg(w2v_max_preds,5))

#### => Not really better

## Let's visualize learned embeddings

Just like in the 1st practical, we propose to visualize learnt items embeddings with the [Tensorflow projector](https://projector.tensorflow.org/).

In [ ]:
# This function saves embeddings (a numpy array) and associated labels into tsv files.

def save_embeddings(embs,dict_label,path="saved_word_vectors"):
    """
    embs is Numpy.array(N,size)
    dict_label is {str(word)->int(idx)} or {int(idx)->str(word)}
    """
    def int_first(k,v):
        if type(k) == int:
            return (k,v)
        else:
            return (v,k)

    np.savetxt(f"{path}_vectors.tsv", embs, delimiter="\t")

    #labels 
    if dict_label:
        sorted_labs = np.array([lab for idx,lab in sorted([int_first(k,v) for k,v in dict_label.items()])])
        print(sorted_labs)
        with open(f"{path}_metadata.tsv","w") as metadata_file:
            for x in sorted_labs: #hack for space
                if len(x.strip()) == 0:
                    x = f"space-{len(x)}"
                    
                metadata_file.write(f"{x}\n")

In [ ]:
vec2title = {i:id2title[int(mid)] for i,mid in enumerate(w2v.wv.index2word)}

In [ ]:
save_embeddings(w2v.wv.vectors,vec2title)

## How to:

- Now, [open this link](https://projector.tensorflow.org/), and select "load".
- look for saved_word_vectors_vectors.tsv and saved_word_vectors_metadata.tsv. 

=> These are respectively, the items latent representations and their labels

## Hyperparameters matter when using Word2Vec for Item recommendation:


> Skip-gram with negative sampling, a popular variant of Word2vec originally designed and tuned to create word embeddings for Natural Language Processing, has been used to create item embeddings with successful applications in recommendation. While these fields do not share the same type of data, neither evaluate on the same tasks, recommendation applications tend to use the same already tuned hyperparameters values, even if optimal hyperparameters values are often known to be data and task dependent. We thus investigate the marginal importance of each hyperparameter in a recommendation setting through large hyperparameter grid searches on various datasets. Results reveal that optimizing neglected hyperparameters, namely negative sampling distribution, number of epochs, subsampling parameter and window-size, significantly improves performance on a recommendation task, and can increase it by an order of magnitude. Importantly, we find that optimal hyperparameters configurations for Natural Language Processing tasks and Recommendation tasks are noticeably different. 

[Hyperparameters matter](https://arxiv.org/abs/1804.04212)

#### It turns out that  hyperparameters are really important for this task: especially the sampling parameter.  Try and learn multiple models to see how the ns_exponent parameter modifies the results:


In [ ]:
# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=train_seq_str,
                                size=50, window=3,               ### here we train a cbow model 
                                min_count=0,                      
                                sample=0.001, ns_exponent=-0.4, workers=10,
                                sg=1, hs=0, negative=15,          ### set sg to 1 to train a sg model => Prod2Vec
                                cbow_mean=0,
                                iter=25)



In [ ]:
pop_preds = get_predictions(predict_pop,last_consumed_item,test_seq,-1)
w2v_preds = get_predictions(predict_w2v,last_consumed_item,test_seq,-1)

In [ ]:
print(1/mrr(pop_preds))
print(1/mrr(w2v_preds))

print(mean_dcg(pop_preds,5))
print(mean_dcg(w2v_preds,5))

print(mean_ndcg(pop_preds,5))
print(mean_ndcg(w2v_preds,5))

## Still got time ? Try making a more clever item selection mechanism:

- You could, for example, cluster items in groups (using k-means) and propose the most popular items of the last seen group